In [15]:
from keybert import KeyBERT
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd
from tqdm import tqdm, trange
from random import randint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
tqdm.pandas()

In [17]:
def morphs(text, noun = True, verb = False, adjective = False, adverb = False):
    tokens = word_tokenize(text)
    poses = pos_tag(tokens, tagset = 'universal')

    return 

[('My', 'PRP$'),
 ('name', 'NN'),
 ('is', 'VBZ'),
 ('Dong', 'NNP'),
 ('Jae', 'NNP'),
 ('Kim', 'NNP'),
 ('.', '.')]

In [21]:
pos_tag(word_tokenize('Why this code does not work?'), tagset = 'universal')

[('Why', 'ADV'),
 ('this', 'DET'),
 ('code', 'NOUN'),
 ('does', 'VERB'),
 ('not', 'ADV'),
 ('work', 'VERB'),
 ('?', '.')]

In [3]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [4]:
df1 = pd.read_csv('./datasets/01/roblox1.csv', index_col = 0)
df2 = pd.read_csv('./datasets/01/roblox2.csv', index_col = 0)
df3 = pd.read_csv('./datasets/01/zepeto.csv', index_col = 0)
df = pd.concat([df1, df2, df3]).reset_index(drop = True)

/tmp/ipykernel_2535483/2625811950.py:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('./datasets/01/roblox1.csv', index_col = 0)
/tmp/ipykernel_2535483/2625811950.py:2: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./datasets/01/roblox2.csv', index_col = 0)


In [5]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,05426917-05d0-4cce-89d7-27fa81c313f3,LT. Silver,https://play-lh.googleusercontent.com/a-/AD_cM...,As a person who has played this game for many ...,4,219,2.581.563,2023-07-02 19:51:21,NaN,NaN,2.581.563
1,1ef0a865-6bb8-4d2b-911c-ac0603ec3091,Hiba Nur,https://play-lh.googleusercontent.com/a/AAcHTt...,"Overall the game is pretty good, but there are...",4,4129,2.581.563,2023-06-27 02:54:39,NaN,NaN,2.581.563
2,1c0c638e-8c94-4421-bd3c-0b13d2f1aed5,Sophia Lewis,https://play-lh.googleusercontent.com/a/AAcHTt...,"Hello, I like on how it is very limitless and ...",3,283,2.581.563,2023-06-28 03:30:25,NaN,NaN,2.581.563
3,b8da4c78-1fd1-4d83-922e-814fb1fc0d72,mamaLo Z,https://play-lh.googleusercontent.com/a-/AD_cM...,"Gotta say, great app! Been playing for years n...",4,108,2.581.563,2023-07-04 12:32:28,NaN,NaN,2.581.563
4,daad7979-0220-4202-a979-3383119e2fff,KitKat Scratcher,https://play-lh.googleusercontent.com/a/AAcHTt...,"I've been playing for years, and this game is ...",4,883,2.578.564,2023-06-28 15:10:06,NaN,NaN,2.578.564
...,...,...,...,...,...,...,...,...,...,...,...
485123,90de0a84-0de0-430e-88c3-73f38afb4142,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,гоё гоё,1,0,NaN,2019-07-05 04:42:29,NaN,NaN,NaN
485124,216320bc-49e0-45f4-9e1f-c006e9f8d17a,Joana angela Lopez,https://play-lh.googleusercontent.com/a/AAcHTt...,Yumica Yumica Yumica Yumica,5,0,NaN,2021-10-12 16:47:32,NaN,NaN,NaN
485125,7f402bb6-22dd-4306-9448-36bfc7835f53,EliD,https://play-lh.googleusercontent.com/a-/AD_cM...,ZEPETO ZEPETO ZEPETO ZEPETO ZEPETO,5,0,NaN,2021-09-08 20:04:37,"Have a lovely time! EliD, Thanks for playing Z...",2021-09-09 01:28:51,NaN
485126,fa33d0ac-54a4-4886-9833-ecbee1bc1211,B Das,https://play-lh.googleusercontent.com/a-/AD_cM...,Hehe Hehe Hehe Hehe Hehe,5,0,NaN,2021-06-14 06:45:39,NaN,NaN,NaN


In [ ]:
df['keybert_keywords'] = df['content'].progress_apply(lambda x : model.extract_keywords(x, top_n = 10))